# Alumno: Gerardo de Miguel González

##  Práctica Remuestreo

### A. Carga y limpieza de los datos

In [1]:
#::GMG::Cargamos los datos
data <- read.csv(file = 'm1965-stats-ii-datos-practica-remuestreo.csv',header = TRUE,sep = ',')

In [2]:
#::GMG::La columna de de fechas (date) está en un formato no compatible con el tipo de fecha
#       DD/MM/YYYY adecuado para una indexación posterior que vamos a necesitar para la selección
#       de los parámetros AVG.Cond. (Conductividad) y AVG.Salinity. (Salinidad) de 2014 ...
#       Ajustamos las fechas
data$date <- as.Date(x = data$date, format='%d/%m/%Y %H:%M')

In [3]:
#::GMG::Vamos a extraer los datos que necesitamos para el remuestreo
myvars <- names(data) %in% c("date", "AVG.Temp.")
data.processed <- data[myvars]
head(data.processed)

date,AVG.Temp.
2014-01-01,3.997746
2014-01-02,4.057667
2014-01-03,4.133612
2014-01-04,4.188685
2014-01-05,4.346172
2014-01-06,4.392661


In [4]:
#::GMG::Ahora vamos a separar los datos de "verano" de 2014 y 2015
# https://stackoverflow.com/questions/9407622/subsetting-a-dataframe-for-a-specified-month-and-year
data.processed.verano <- subset(
           x = data.processed, 
           #data.processed$date >= as.Date('2014-07-01') & 
           #data.processed$date <= as.Date('2014-09-31')
           format.Date(data.processed$date, '%m') >= '07' &
           format.Date(data.processed$date, '%m') <= '09'
           )
head(data.processed.verano)

,date,AVG.Temp.
182,2014-07-01,15.41396
183,2014-07-02,14.70326
184,2014-07-03,14.76020
185,2014-07-04,15.12685
186,2014-07-05,15.19523
187,2014-07-06,15.24690


In [5]:
#::GMG::Ahora selecciono y limpio el vector (muestra) de temperatura de 2014 y 2015
#       Hay que limpiar los NaNs con na.omit()
# https://www.statmethods.net/input/missingdata.html
x.verano <- na.omit(data.processed.verano$AVG.Temp.)
paste('Tamaño de la muestra:',length(x.verano))

[1] "Tamaño de la muestra: 182"

### B. Media y desviación muestral

**::GMG::** Cálculo de la media y deviación estándar muestral (estadísticos/estimadores)

In [6]:
#::GMG::Usamos las funciones que nos proporciona R
media.x.verano <- mean(x.verano)
sd.x.verano <- sd(x.verano)
paste('Media muestral:',round(media.x.verano,3))
paste('Desv. Estándar muestral:', round(sd.x.verano,3))

[1] "Media muestral: 18.066"

[1] "Desv. Estándar muestral: 1.971"

In [7]:
#::GMG::Yo por completar también calculo la varianza  muestral
paste('Varianza muestral:', round(var(x.verano),3))

[1] "Varianza muestral: 3.884"

### C. Bootstrap

#### Algoritmo

Crea una función que reciba un vector de números ‘x’ (la muestra original) y un número
natural N, que genere una matriz que tenga N columnas siendo cada una de estas, una de las
muestras de bootstrap.

**::GMG::** El algoritmo de Bootstrap lo implenté en el **Ejercicio 8** de clase con Pablo Ruiz (referencia a *transparencia 7*, presentación `chapter 8` de clase)

In [8]:
#::GMG::Hago el experimento replicable
set.seed(1)

In [9]:
#::GMG::Mi función, con replicate() y sample(), genero una matriz de length(v)xN
# https://www.rdocumentation.org/packages/base/versions/3.5.2/topics/lapply
# replicate() is a wrapper for the common use of sapply() for repeated evaluation of 
# an expression (which will usually involve random number generation).
bootstrap <- function(v, N) {
    return (
             replicate(n = N, expr = sample(x = v, replace = TRUE))
           )
}

In [10]:
#::GMG::Prueba de concepto
x <- c(5,3,1,7,9,2)
bootstrap(x,10)

3,2,9,1,3,1,9,9,9,5
1,7,1,9,1,7,5,7,9,5
7,7,9,2,5,1,9,7,1,3
2,5,1,3,1,3,1,9,2,7
3,3,9,7,2,9,9,5,1,7
2,3,2,5,1,9,7,1,3,1


In [11]:
#::GMG::Como dicen en rdocumentation, replicate() es un wrapper de sapply()
bootstrap_2 <- function(v, N) {
    return (sapply(1:N, function(i) sample(v, replace=T)))
}

In [12]:
#::GMG::Prueba de concepto
bootstrap_2(x,10)

2,1,1,9,9,3,1,3,2,5
3,9,1,2,3,5,1,2,7,5
1,5,1,1,9,7,9,7,2,9
3,2,2,9,5,2,7,3,9,5
7,1,2,1,3,9,7,5,1,1
3,2,1,3,5,9,1,1,1,7


#### Aplicación a la media de temperaturas del ambalse

**Tarea 1**: Obtener, siguiendo el algoritmo Bootstrap, una matriz de 1000 muestras por columna:  

In [13]:
#::GMG::Hago el experimento replicable
set.seed(2)

In [14]:
#::GMG::Una matriz con los 1000 samples Boostrap de la muestra de tamaño 182 seleccionada del dataset
theta.B.M <- bootstrap(v = x.verano, N = 1000)
dim(theta.B.M)

[1]  182 1000

**Tarea 2:** Calcular con el resampling anterior:

- La media y la desviación estándar de la media y compararlas con las obtenidas anteriormente.

$SD = \sqrt {\frac {1}{M} \sum \limits_{i=1}^M \left( \bar {\Theta^{B}_{i}} - \bar \Theta \right)^2}$ 

**::GMG::** Entiendo que lo que pide en esta *tarea 2* es comparar los valores de media de medias de Bootstrap y desviación estándar de medias de Bootstrap (usando la fórmula SD), con la media muestral y desviación estándar muestral obtenidas en el *aparatado B*. 

**::nota::** SD se puede considerar como una estimación del error estándar que cometen los estadísticos calculados en el apartado B (deducido de los apuntes de Pablo Ruiz en clase), aunque la notación es diferente:

$SD = \sqrt {\frac {1}{N-1} \sum \limits_{i=1}^N \left( \bar {\Theta^B_{i}} - \bar \Theta \right)^2}$

**::DUDAS::** 

- No entiendo la primera parte de la frase. La *media de la medias* del Bootstrap entiendo que se calcula de la siguiente manera para un Bootstrap de tamaño M:

(1) $\bar \theta^{B} = \frac {1}{M} \sum \limits_{i=1}^M \bar \theta_{i}^{B}$

**::nota::** esto es lo que Pablo Ruiz llama $\bar {\bar \Theta}$ en la transparencia 6 donde se explica la técnica del Bootstrap.

y la *desviación estándar de la media de medias* del Bootstrap creo que se calcula (estima) de la siguiente manera:

(2) $\sigma_{\bar \theta^{B}} = \sqrt {\frac {1}{M} \sum \limits_{i=1}^M \left( \bar \theta_{i}^{B} - \bar \theta^{B} \right)^2}$

o si se quiere de forma insesgada:

(3) $\sigma_{\bar \theta^{B}} = \sqrt {\frac {1}{M-1} \sum \limits_{i=1}^M \left( \bar \theta_{i}^{B} - \bar \theta^{B} \right)^2}$


- ¿Tiene esto que ver con la aplicación del Bootstrap que explicó Pablo Ruiz en las transparencias 8 y 9 del capítulo (Chapter 8) de introducción a las técnicas de Bootstrap?
  - Aplicación 1: Aproximación del error standard de un estadístico (transparencia 8)
  - Aplicación 2: Aproximación del sesgo de un estadístico (transparencia 9)

- ¿Aquí se puede considerar los estadísticos *media muestral* y *desviación estándar muestral* de la temperatura del agua del embalse que hemos calculado en el apartado anterior como los *estimadores* sobre los que queremos calcular las aproximación (en concreto la *Applicación 1*) con el algoritmo Bootstrap?

**::GMG::** En el caso de que sea la *aplicación 1*, entonces lo que se pide con la fórmula del enunciado es calcular la aproximación del error estándar de *los estadísticos como estimadores* que hemos calculado en el primer apartado para los *parámetros* media y desviación típica de la variable aleatoria continua temperatura del embalse sin hacer ninguna asunción sobre cómo se distribuye esa variable. 

**::GMG::** La explicación de Pablo en clase de la *aplicación 1* de Bootstrap partía de la existencia de un parámetro $\Theta$ sobre el que hemos obtenido un estimador $\bar \Theta$. Queremos conocer la *desviación estándar* de ese estimador, es decir, su *error estándar* (*standard error*). Para eso tendríamos que hacer M experimentos y usar la siguiente fórmula para *estimarlo* (de forma insesgada):

$SD_{\bar \Theta} = \sqrt {\frac {1}{M-1} \sum \limits_{i=1}^M \left( \bar \Theta_{i} - \Theta \right)^2}$

Pero en lugar de eso (hacer experimentos es muy caro o son *irrepetibles*) lo que se hace es un *resampling* de la muestra original obteniendo M muestras resampleadas siguiendo el algoritmo de *Bootstrap*, aplicando la misma fórmula anterior con $\bar \Theta_{i} → \bar \Theta_{i}^B$. Ahora según un famoso paper:

- Efron, B. (1979). "Bootstrap methods: Another look at the jackknife". The Annals of Statistics. 7 (1): 1–26. doi:10.1214/aos/1176344552.

la variable aleatoria de Bootstrap $\bar \Theta^B$ se distribuye de modo que:

$\bar \Theta^B - \bar \Theta \approx \bar \Theta - \Theta$

**::nota::** donde $\bar \Theta^B$ (la media de las medias) es lo que yo he puesto en la fórmula (1) 

y se obtiene sustituyendo la fórmula del enunciado (versión insesgada que viene en los apuntes de Pablo Ruiz):

$SD_{\bar \Theta} = \sqrt {\frac {1}{M-1} \sum \limits_{i=1}^M \left( \bar \Theta_{i}^B - \bar \Theta \right)^2}$

In [15]:
#::GMG::Calculamos (1)
#theta.bar.B.i <- apply(X = theta.B.M,MARGIN = 2,FUN = mean)
#theta.bar.B <- sum(theta.bar.B.i)/length(theta.bar.B.i)
#::nota::lo hago compacto y eficiente con apply()
# https://www.rdocumentation.org/packages/base/versions/3.5.2/topics/apply
theta.bar.B <- mean(apply(X = theta.B.M, MARGIN = 2, FUN = mean))
paste('Resultado media Bootstrap (1):', round(theta.bar.B,4))
paste('Resultado anterior (media muestral):',round(media.x.verano,4))

[1] "Resultado media Bootstrap (1): 18.0662"

[1] "Resultado anterior (media muestral): 18.0659"

**::GMG::** Comentarios

El segundo estadístico mostrado (*media muestral*) es un estimador del parámetro media poblacional de la variable aleatoria continua que representa la temperatura del embalse de la que desconocemos distribución y parámetros. Se ha calculado de una sola muestra de tamaño *182*. Es también una variable aleatoria con su distribución y parámetros desconocidos. 

El primer estadístico es una media de medias de 1000 experimentos (1000 realizaciones de muestras de tamaño 182) obtenidos por resampling usando el algoritmo Bootstrap. También es un estimador del parámetro media poblacional de la variable aleatoria continua que representa la temperatura del embalse. Y es también una variable aleatoria con su distribución y parámetros desconocidos. 

In [16]:
#::GMG::Estimación del error estándar del estimador de la media
#::nota::el vector de las 1000 medias (de cada muestra) de bootstrap
theta.bar.B.i <- apply(X = theta.B.M,MARGIN = 2,FUN = mean)
#::nota::implementación de la fórmula (2) con sum() y length()
sd.err.mean <- sqrt(
    sum((theta.bar.B.i - media.x.verano)^2)/length(theta.bar.B.i)
)
paste('Resultado sd error media:',round(sd.err.mean,5))

[1] "Resultado sd error media: 0.14021"

In [17]:
#::GMG::Estimación del error estándar del estimador de la desviación estándar
theta.bar.B.i <- apply(X = theta.B.M,MARGIN = 2,FUN = sd)
sd.err.desv <- sqrt(
    sum((theta.bar.B.i - sd.x.verano)^2)/length(theta.bar.B.i)
)
paste('Resultado sd error desv. estand.:',round(sd.err.desv,5))

[1] "Resultado sd error desv. estand.: 0.14562"

**::DUDA::** ¿Cómo se interpreta ésto?

### D. Jackknife

#### Algoritmo

In [18]:
#::GMG::Implemento mi función que genera jackknife "N-1"
#::nota::Misma implementación que el Ejercicio 8 de clase
genera.jackknife <- function(v) {
    M <- matrix(nrow = length(v)-1, ncol = length(v))
    for (i in 1:length(v)) {
        M[,i] <- v[-i]
    }
    return(M)
}

In [19]:
#::GMG::Ejemplo
matriz.jackknife <-genera.jackknife(x)
dim(matriz.jackknife)
print(matriz.jackknife)

[1] 5 6

     [,1] [,2] [,3] [,4] [,5] [,6]
[1,]    3    5    5    5    5    5
[2,]    1    1    3    3    3    3
[3,]    7    7    7    1    1    1
[4,]    9    9    9    9    7    7
[5,]    2    2    2    2    2    9


#### Aplicación a la media de temperaturas

In [20]:
#::GMG::repito el proceso con Jackknife
matriz.replicas.jk <- genera.jackknife(v = x.verano)
medias.jk <- apply(X = matriz.replicas.jk,MARGIN = 2,FUN = mean)
str(matriz.replicas.jk)
str(medias.jk)

 num [1:181, 1:182] 14.7 14.8 15.1 15.2 15.2 ...
 num [1:182] 18.1 18.1 18.1 18.1 18.1 ...


In [23]:

media.jk <- mean(medias.jk)
paste('Estimación de la media jackknife:', round(media.jk,5))
paste('Resultado anterior (media muestral):',round(media.x.verano,4))

[1] "Estimación de la media jackknife: 18.06591"

[1] "Resultado anterior (media muestral): 18.0659"

In [24]:
#::GMG::En el cálculo de la sd del jackknife hay que hacer una "corrección"
# https://en.wikipedia.org/wiki/Jackknife_resampling#Estimation
sigma.jk <- sqrt((length(medias.jk) - 1)*mean((medias.jk - media.jk)^2))
paste('Estimación de error estándar estimador media por jackknife',round(sigma.jk,5))

[1] "Estimación de error estándar estimador media por jackknife 0.14609"